# bisect — Array bisection algorithm

This module provides support for maintaining a list in sorted order without having to sort the list after each insertion. For long lists of items with expensive comparison operations, this can be an improvement over the more common approach. The module is called bisect because it uses a basic bisection algorithm to do its work. The source code may be most useful as a working example of the algorithm (the boundary conditions are already right!).

The following functions are provided:

bisect.**bisect_left**(*a, x, lo=0, hi=len(a)*)

Locate the insertion point for x in a to maintain sorted order. The parameters lo and hi may be used to specify a subset of the list which should be considered; by default the entire list is used. If x is already present in a, the insertion point will be before (to the left of) any existing entries. The return value is suitable for use as the first parameter to list.insert() assuming that a is already sorted.

The returned insertion point i partitions the array a into two halves so that all(val < x for val in a[lo:i]) for the left side and all(val >= x for val in a[i:hi]) for the right side.

bisect.**bisect_right**(*a, x, lo=0, hi=len(a)*)

bisect.**bisect**(*a, x, lo=0, hi=len(a)*)

Similar to bisect_left(), but returns an insertion point which comes after (to the right of) any existing entries of x in a.

The returned insertion point i partitions the array a into two halves so that all(val <= x for val in a[lo:i]) for the left side and all(val > x for val in a[i:hi]) for the right side.

bisect.**insort_left**(*a, x, lo=0, hi=len(a)*)

Insert x in a in sorted order. This is equivalent to a.insert(bisect.bisect_left(a, x, lo, hi), x) assuming that a is already sorted. Keep in mind that the O(log n) search is dominated by the slow O(n) insertion step.

bisect.**insort_right**(*a, x, lo=0, hi=len(a)*)

bisect.**insort**(*a, x, lo=0, hi=len(a)*)

Similar to insort_left(), but inserting x in a after any existing entries of x.

## Searching Sorted Lists

The above bisect() functions are useful for finding insertion points but can be tricky or awkward to use for common searching tasks. The following five functions show how to transform them into the standard lookups for sorted lists:


In [6]:
import random
from random import randint
from bisect import *

In [1]:
def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    raise ValueError

def find_lt(a, x):
    'Find rightmost value less than x'
    i = bisect_left(a, x)
    if i:
        return a[i-1]
    raise ValueError

def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect_right(a, x)
    if i:
        return a[i-1]
    raise ValueError

def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect_right(a, x)
    if i != len(a):
        return a[i]
    raise ValueError

def find_ge(a, x):
    'Find leftmost item greater than or equal to x'
    i = bisect_left(a, x)
    if i != len(a):
        return a[i]
    raise ValueError

In [7]:
random.seed(42)
my_list = []
for _ in range(10):
    insort_left(my_list, randint(1,100))
my_list

[4, 14, 15, 18, 29, 32, 36, 82, 95, 95]

In [9]:
# index(a, x)
try:
    print(index(my_list, 18))
    print(index(my_list, 100))
except ValueError as e:
    print(repr(e))

3
ValueError()


In [11]:
# find_lt(a, x)
try:
    print(find_lt(my_list, 18))
    print(find_lt(my_list, 4))
except ValueError as e:
    print(repr(e))

15
ValueError()


In [16]:
# find_le(a, x)
try:
    print(find_le(my_list, 18))
    print(find_le(my_list, 4))
except ValueError as e:
    print(repr(e))

18
4


In [19]:
# find_gt(a, x)
try:
    print(find_gt(my_list, 18))
    print(find_gt(my_list, 95))
except ValueError as e:
    print(repr(e))

29
ValueError()


In [20]:
# find_ge(a, x)
try:
    print(find_ge(my_list, 18))
    print(find_ge(my_list, 95))
except ValueError as e:
    print(repr(e))

18
95


## Other Examples

The bisect() function can be useful for numeric table lookups. This example uses bisect() to look up a letter grade for an exam score (say) based on a set of ordered numeric breakpoints: 90 and up is an ‘A’, 80 to 89 is a ‘B’, and so on:

In [25]:
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

Unlike the sorted() function, it does not make sense for the bisect() functions to have key or reversed arguments because that would lead to an inefficient design (successive calls to bisect functions would not “remember” all of the previous key lookups).

Instead, it is better to search a list of precomputed keys to find the index of the record in question:

In [35]:
data = [('red', 5), ('blue', 1), ('yellow', 8), ('black', 0)]
data.sort(key=lambda r: r[1])
keys = [r[1] for r in data]         # precomputed list of keys
print(data[bisect_left(keys, 0)])
print(data[bisect_left(keys, 1)])
print(data[bisect_left(keys, 5)])
print(data[bisect_left(keys, 8)])

('black', 0)
('blue', 1)
('red', 5)
('yellow', 8)


**See also**:

[SortedCollection](https://code.activestate.com/recipes/577197-sortedcollection/) recipe that uses bisect to build a full-featured collection class with straight-forward search methods and support for a key-function. The keys are precomputed to save unnecessary calls to the key function during searches.
 